In [8]:
import pandas as pd
import os
os.getcwd()

'C:\\Users\\Atchyuta\\Downloads'

In [ ]:
# Combine all the the text files along with tag into CSV file.

# Using readlines()
file1 = open('./metadata/mapping_conv_topic.train.txt', 'r')
Lines = file1.readlines()
import re 
count = 0
# Strips the newline character
final_list=[]
for line in Lines:
    count += 1
    file_tag=re.findall('"([^"]*)"', line)
    file_no=line.split(' ')[0]
    with open('trans.'+file_no+'.txt', 'r') as file:
        file_data = file.read().replace('\n', ' ')
        file_data=re.sub("\d+\.\d+ ", '',file_data)
    final_list.append({"file_name":'trans.'+file_no+'.txt',"file_content":file_data,"file_tag":file_tag[0]})
    #print("Line{}: {}".format(count, line.strip()))

import csv
keys = final_list[0].keys()
with open('train_data_tagging.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(final_list)


In [68]:
#!pip install gensim

  Created wheel for smart-open: filename=smart_open-4.2.0-py3-none-any.whl size=109632 sha256=353dfdabcc56405a11018dae28410c5e18beb5444d672f9f0a9097371c0f84ec
  Stored in directory: c:\users\atchyuta\appdata\local\pip\cache\wheels\25\88\e3\7cd51a6379cac37213cac47545a27688782752ff66351b953d
Successfully built smart-open


In [9]:
#Reading the file
data=pd.read_csv('C:\\Users\\Atchyuta\\Downloads\\train_data_tagging_wo_duration.csv')

In [10]:
data.head()

,file_name,file_content,file_tag
0,trans.2023.txt,Agent [silence] Customer all right Agent okay ...,Credit Card
1,trans.2061.txt,Agent [noise] Customer [noise] Agent [silence]...,Credit Card
2,trans.2067.txt,Agent [noise] Customer [noise] Agent well um w...,Credit Card
3,trans.2163.txt,Agent [silence] Customer okay i was trying to ...,Credit Card
4,trans.2313.txt,Agent [noise] Customer [silence] Agent [silenc...,Credit Card


In [11]:
from sklearn import model_selection, preprocessing, metrics


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.model_selection import train_test_split

from keras.layers import concatenate
from sklearn.metrics import mean_squared_error
import nltk
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize          
from nltk.corpus import stopwords
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Input, Bidirectional,TimeDistributed
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [13]:
data['file_tag'].unique()

array(['Credit Card', 'Bank Bailout', 'Taxes', 'Job Benefits',
       'Family Finance', 'Budget'], dtype=object)

In [14]:
# Tokenizing and cleaning Each sentence
def buildVocabulary(sentence):
    text = list(set(sentence))
    tokenizer = Tokenizer(lower=False, split=' ')
    tokenizer.fit_on_texts(text)
    return tokenizer
def cleanSentence(sentence):
     sentence_clean= re.sub("[^a-zA-Z0-9]"," ", str(sentence))
     sentence_clean = sentence_clean.lower()
     tokens = word_tokenize(sentence_clean)
     stop_words = set(stopwords.words("english"))
     sentence_clean_words = [w for w in tokens if not w in stop_words]
     return ' '.join(sentence_clean_words)

In [15]:
nltk.download('punkt')
nltk.download('stopwords')

#clean the sentence 
data['clean_file_content'] = list(map(cleanSentence, data['file_content']))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Atchyuta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Atchyuta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
corpus_text1 = '\n'.join(data['clean_file_content'])
sentences = corpus_text1.split('\n')

In [17]:
tokenizer = buildVocabulary(sentences)


In [18]:
#Saving Tokenizer for Inference purpose
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
vocab_size = len(tokenizer.word_index) + 1


In [13]:
vocab_size

8216

'2.1.6'

In [19]:
data['word_count'] = data['clean_file_content'].apply(lambda x: len(str(x).split(" ")))
#incidents_data['word_count'] = incidents_data['short_description'].apply(lambda x: len(str(x).split(" ")))
data[['clean_file_content','word_count']].head(11)

,clean_file_content,word_count
0,agent silence customer right agent okay custom...,1320
1,agent noise customer noise agent silence custo...,1676
2,agent noise customer noise agent well um credi...,576
3,agent silence customer okay trying get childre...,431
4,agent noise customer silence agent silence cus...,1080
5,agent silence customer right agent well like t...,1314
6,agent noise customer noise agent silence custo...,1329
7,agent silence customer noise customer okay um ...,1000
8,agent noise customer noise customer right agen...,999
9,agent silence customer noise customer probably...,1484


In [43]:
data['word_count'].max()

1676

In [42]:
data['word_count'].min()

431

In [20]:
#To Genrate Train Sequences
def getTrainSequences(sentence, tokenizer,seq_maxlen):
    sent1 = tokenizer.texts_to_sequences(sentence)
    #sent_maxlen = max([len(s) for s in sent1])
    #seq_maxlen = max([sent_maxlen])
    return np.array(pad_sequences(sent1, maxlen=seq_maxlen))

In [21]:
#get sequences for each sentence pair tuple
train_data_need = getTrainSequences(sentences, tokenizer,seq_maxlen=1680)

In [24]:
train_data_need.shape


(240, 1680)

In [29]:
train_data_need[0]

array([0, 0, 0, ..., 8, 1, 3])

In [30]:
tokenizer


8215


In [17]:
word_embed_size = 100


In [18]:
#100d Glove vector file
glove_file = 'C:\\Users\\Atchyuta\\Downloads\\glove.6B.100d.txt'

In [19]:
#Loading Glove file
def loadGloveWordEmbeddings(glove_file):
    embedding_vectors = {}
    f = open(glove_file,encoding='utf8')
    for line in f:
        values = line.split()
        word = values[0]
        value = np.asarray(values[1:], dtype='float32')
        embedding_vectors[word] = value
    f.close()
    return embedding_vectors

In [20]:
# Creating embedding matrix for each word based on Glove vector
def getEmbeddingWeightMatrix(embedding_vectors, word2idx):    
    embedding_matrix = np.zeros((len(word2idx)+1, word_embed_size))
    for word, i in word2idx.items():
        embedding_vector = embedding_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [21]:
embedding_vectors = loadGloveWordEmbeddings(glove_file)

In [22]:
import pickle

# saving
with open('embedding_vectors.pickle', 'wb') as handle:
    pickle.dump(embedding_vectors, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
print(len(embedding_vectors))


400000


In [31]:
tokenizer.word_index

{'agent': 1,
 'customer': 2,
 'silence': 3,
 'uh': 4,
 'know': 5,
 'yeah': 6,
 'um': 7,
 'laughter': 8,
 'noise': 9,
 'well': 10,
 'like': 11,
 'right': 12,
 'think': 13,
 'hum': 14,
 'huh': 15,
 'get': 16,
 'oh': 17,
 'really': 18,
 'one': 19,
 'money': 20,
 'pay': 21,
 'got': 22,
 'people': 23,
 'going': 24,
 'lot': 25,
 'mean': 26,
 'much': 27,
 'things': 28,
 'would': 29,
 'good': 30,
 'go': 31,
 'something': 32,
 'kind': 33,
 'credit': 34,
 '1': 35,
 'time': 36,
 'work': 37,
 'okay': 38,
 'thing': 39,
 'guess': 40,
 'tax': 41,
 'see': 42,
 'vocalized': 43,
 'way': 44,
 'taxes': 45,
 'dollars': 46,
 'use': 47,
 'year': 48,
 'say': 49,
 'years': 50,
 'two': 51,
 'budget': 52,
 'little': 53,
 'card': 54,
 'month': 55,
 'yes': 56,
 'back': 57,
 'pretty': 58,
 'even': 59,
 'take': 60,
 'make': 61,
 'every': 62,
 'could': 63,
 'us': 64,
 'need': 65,
 'benefits': 66,
 'sure': 67,
 'want': 68,
 'put': 69,
 'five': 70,
 'company': 71,
 'probably': 72,
 'paying': 73,
 'anything': 74,
 'card

In [32]:
file = open("tokenizer.pickle",'rb')
tokenizer1 = pickle.load(file)
print("len(tokenizer.word_index) + 1:",len(tokenizer1.word_index) + 1)
file.close()

len(tokenizer.word_index) + 1: 8216


In [33]:
tokenizer1.word_index

{'agent': 1,
 'customer': 2,
 'silence': 3,
 'uh': 4,
 'know': 5,
 'yeah': 6,
 'um': 7,
 'laughter': 8,
 'noise': 9,
 'well': 10,
 'like': 11,
 'right': 12,
 'think': 13,
 'hum': 14,
 'huh': 15,
 'get': 16,
 'oh': 17,
 'really': 18,
 'one': 19,
 'money': 20,
 'pay': 21,
 'got': 22,
 'people': 23,
 'going': 24,
 'lot': 25,
 'mean': 26,
 'much': 27,
 'things': 28,
 'would': 29,
 'good': 30,
 'go': 31,
 'something': 32,
 'kind': 33,
 'credit': 34,
 '1': 35,
 'time': 36,
 'work': 37,
 'okay': 38,
 'thing': 39,
 'guess': 40,
 'tax': 41,
 'see': 42,
 'vocalized': 43,
 'way': 44,
 'taxes': 45,
 'dollars': 46,
 'use': 47,
 'year': 48,
 'say': 49,
 'years': 50,
 'two': 51,
 'budget': 52,
 'little': 53,
 'card': 54,
 'month': 55,
 'yes': 56,
 'back': 57,
 'pretty': 58,
 'even': 59,
 'take': 60,
 'make': 61,
 'every': 62,
 'could': 63,
 'us': 64,
 'need': 65,
 'benefits': 66,
 'sure': 67,
 'want': 68,
 'put': 69,
 'five': 70,
 'company': 71,
 'probably': 72,
 'paying': 73,
 'anything': 74,
 'card

In [23]:
embedding_weight_matrix = getEmbeddingWeightMatrix(embedding_vectors, tokenizer.word_index)


In [24]:
import pickle

# saving
with open('embedding_weight_matrix.pickle', 'wb') as handle:
    pickle.dump(embedding_weight_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
embedding_weight_matrix.shape

(8216, 100)

In [26]:
#Doing Label Encoding the target variable
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data["file_tag_encoder"] = lb_make.fit_transform(data.file_tag.astype(str))
data[["file_tag", "file_tag_encoder"]].head(11)

,file_tag,file_tag_encoder
0,Credit Card,2
1,Credit Card,2
2,Credit Card,2
3,Credit Card,2
4,Credit Card,2
5,Credit Card,2
6,Credit Card,2
7,Credit Card,2
8,Credit Card,2
9,Credit Card,2


In [27]:
#Doing Stratify spit the as the data also imbalanced
train_x, Test_x, train_y, Test_y = model_selection.train_test_split(train_data_need, data['file_tag_encoder'],test_size=0.20, random_state=42, stratify=data['file_tag_encoder'])


In [28]:
#Splitting data into test and Validation
val_x, test_x, val_y, test_y = model_selection.train_test_split(Test_x, Test_y,test_size=0.20, random_state=42)

In [29]:
from keras.utils import np_utils
y1_train1 = np_utils.to_categorical(train_y)
y1_test = np_utils.to_categorical(test_y)
y1_val=np_utils.to_categorical(val_y)

In [30]:
##Attention Layer

import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model

from sklearn.metrics import roc_auc_score


def dot_product(x, kernel):
    
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):    

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [31]:
seq_maxlen=1680
word_embed_size = 100


In [47]:
y1_train1.shape[1]

6

In [32]:
train_x.shape

(192, 1680)

In [33]:
##Model Building on LSTM and  Attention layer
input = Input(shape=(train_x.shape[1],))

inner = Embedding(input_dim=vocab_size, output_dim=word_embed_size, 
                   input_length=seq_maxlen, weights=[embedding_weight_matrix], 
                   trainable = True) (input)

inner = LSTM(100, return_sequences=True)(inner)
inner = Dropout(0.3)(inner)
# compute importance for each step
#inner = TimeDistributed(Dense(100))(inner) 
words_att = AttentionWithContext()(inner)

inner = Dense(100, activation='relu')(words_att)
#inner = Dense(40, activation='relu')(inner)
output = Dense(y1_train1.shape[1], activation='softmax')(inner)

model_a = Model(inputs = input, outputs = output)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [50]:
model_a.save('my_lstm_model.h5')

In [51]:
import keras

model_a1 = keras.models.load_model('my_lstm_model.h5', custom_objects={'AttentionWithContext':AttentionWithContext()})

D:\Anaconda3\envs\keras-tf\lib\site-packages\keras\models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [52]:
print(model_a1.summary())
model_a1.compile(Adam(lr=0.01), 'categorical_crossentropy', metrics=['accuracy'])
#model_a.load_weights('/var/www/projects/nehru/text_classification/code/model_incidents_attention_lstm_word_vec_100d_n1.h5')

save_weights = ModelCheckpoint('model_doc_tagging_attention_lstm_word_vec_100d_new.h5', monitor='val_loss', save_best_only=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1680)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1680, 100)         821600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1680, 100)         80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1680, 100)         0         
_________________________________________________________________
attention_with_context_1 (At (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 606       
Total para

In [53]:
from keras.models import model_from_json
model_json = model_a1.to_json()
with open("model_incident_classification_lstm_attention.json", "w") as json_file:
    json_file.write(model_json)

In [54]:
history = model_a1.fit(x=train_x, y=y1_train1, batch_size=32,
          epochs=30, validation_data=(val_x, y1_val), callbacks=[save_weights])

Instructions for updating:
Use tf.cast instead.
Train on 192 samples, validate on 38 samples
Epoch 1/30
192/192 [==============================] - 9s 47ms/step - loss: 1.8574 - acc: 0.1302 - val_loss: 1.7234 - val_acc: 0.2895
Epoch 2/30
192/192 [==============================] - 8s 41ms/step - loss: 1.6162 - acc: 0.3073 - val_loss: 1.5576 - val_acc: 0.2632
Epoch 3/30
192/192 [==============================] - 8s 42ms/step - loss: 1.3082 - acc: 0.5312 - val_loss: 1.2284 - val_acc: 0.6316
Epoch 4/30
192/192 [==============================] - 8s 43ms/step - loss: 0.9096 - acc: 0.6823 - val_loss: 0.8917 - val_acc: 0.7632
Epoch 5/30
192/192 [==============================] - 8s 44ms/step - loss: 0.5690 - acc: 0.8333 - val_loss: 0.3783 - val_acc: 0.8947
Epoch 6/30
192/192 [==============================] - 8s 44ms/step - loss: 0.3118 - acc: 0.9167 - val_loss: 0.3332 - val_acc: 0.9211
Epoch 7/30
192/192 [==============================] - 8s 44ms/step - loss: 0.1982 - acc: 0.9219 - val_loss: 0

# Predicting on Test Data

In [55]:
predicted_lstm=model_a1.predict(test_x)
predicted_lstm=predicted_lstm.argmax(axis=-1)
y1_test=y1_test.argmax(axis=-1)

In [56]:
accuracy1=np.mean(predicted_lstm==y1_test)


In [57]:
accuracy1

1.0

In [59]:
y1_test

array([3, 2, 4, 3, 5, 4, 3, 5, 4, 5], dtype=int64)

In [58]:
predicted_lstm

array([3, 2, 4, 3, 5, 4, 3, 5, 4, 5], dtype=int64)

# GRU

In [67]:
##Model Building on LSTM and  Attention layer
input = Input(shape=(train_x.shape[1],))

inner = Embedding(input_dim=vocab_size, output_dim=word_embed_size, 
                   input_length=seq_maxlen, weights=[embedding_weight_matrix], 
                   trainable = True) (input)

inner = GRU(100, return_sequences=True)(inner)
inner = Dropout(0.3)(inner)
# compute importance for each step
#inner = TimeDistributed(Dense(100))(inner) 
words_att = AttentionWithContext()(inner)

inner = Dense(100, activation='relu')(words_att)
#inner = Dense(40, activation='relu')(inner)
output = Dense(y1_train1.shape[1], activation='softmax')(inner)

model_a = Model(inputs = input, outputs = output)

In [68]:
model_a.save('my_GRU_model.h5')

In [69]:
import keras

model_a1 = keras.models.load_model('my_GRU_model.h5', custom_objects={'AttentionWithContext':AttentionWithContext()})

In [70]:
print(model_a1.summary())
model_a1.compile(Adam(lr=0.01), 'categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1680)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1680, 100)         821600    
_________________________________________________________________
gru_3 (GRU)                  (None, 1680, 100)         60300     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1680, 100)         0         
_________________________________________________________________
attention_with_context_9 (At (None, 100)               10200     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 606       
Total para

In [63]:

#model_a.load_weights('/var/www/projects/nehru/text_classification/code/model_incidents_attention_lstm_word_vec_100d_n1.h5')

save_weights = ModelCheckpoint('model_doc_tagging_attention_GRU_glove_vec_100d_new.h5', monitor='val_loss', save_best_only=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1680)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1680, 100)         821600    
_________________________________________________________________
gru_2 (GRU)                  (None, 1680, 100)         60300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1680, 100)         0         
_________________________________________________________________
attention_with_context_7 (At (None, 100)               10200     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 606       
Total para

In [64]:
from keras.models import model_from_json
model_json = model_a1.to_json()
with open("model_incident_classification_gru_attention.json", "w") as json_file:
    json_file.write(model_json)

In [38]:
history = model_a1.fit(x=train_x, y=y1_train1, batch_size=32,
          epochs=30, validation_data=(val_x, y1_val), callbacks=[save_weights])

Instructions for updating:
Use tf.cast instead.
Train on 192 samples, validate on 38 samples
Epoch 1/30
192/192 [==============================] - 9s 47ms/step - loss: 1.7419 - acc: 0.2656 - val_loss: 1.6465 - val_acc: 0.2105
Epoch 2/30
192/192 [==============================] - 8s 40ms/step - loss: 1.2736 - acc: 0.5417 - val_loss: 1.2453 - val_acc: 0.6316
Epoch 3/30
192/192 [==============================] - 8s 42ms/step - loss: 0.8104 - acc: 0.7188 - val_loss: 1.1854 - val_acc: 0.6579
Epoch 4/30
192/192 [==============================] - 8s 42ms/step - loss: 0.6942 - acc: 0.8281 - val_loss: 0.5399 - val_acc: 0.8158
Epoch 5/30
192/192 [==============================] - 9s 47ms/step - loss: 0.4071 - acc: 0.8750 - val_loss: 0.4024 - val_acc: 0.8947
Epoch 6/30
192/192 [==============================] - 8s 44ms/step - loss: 0.1705 - acc: 0.9479 - val_loss: 0.4987 - val_acc: 0.8158
Epoch 7/30
192/192 [==============================] - 8s 44ms/step - loss: 0.0714 - acc: 0.9844 - val_loss: 0

# Predicting on Test Data

In [86]:
predicted_lstm=model_a1.predict(test_x)
predicted_lstm=predicted_lstm.argmax(axis=-1)
y1_test=y1_test.argmax(axis=-1)

In [87]:
accuracy1=np.mean(predicted_lstm==y1_test)


In [88]:
y1_test

array([3, 2, 4, 3, 5, 4, 3, 5, 4, 5], dtype=int64)

In [89]:
accuracy1

1.0

In [90]:
predicted_lstm

array([3, 2, 4, 3, 5, 4, 3, 5, 4, 5], dtype=int64)

In [78]:
y1_test

array([3, 2, 4, 3, 5, 4, 3, 5, 4, 5], dtype=int64)

In [41]:
#Flask Code


import os
import sys
import logging
from flask import Flask, request, jsonify
from flask import Flask, render_template, request
from werkzeug import secure_filename
import re
import flask
import os
from sklearn.model_selection import train_test_split

from keras.layers import concatenate
from sklearn.metrics import mean_squared_error
import nltk
import pandas as pd
import numpy as np
from nltk import word_tokenize          
from nltk.corpus import stopwords
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Input, Bidirectional,TimeDistributed
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
from sklearn.metrics import roc_auc_score
import pickle
import tensorflow as tf
#graph = tf.get_default_graph()



# Define the app
app = Flask(__name__)


# Load the model
##Attention Layer

def dot_product(x, kernel):
    
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):    

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

import keras
global model_n1
global graph
graph = tf.get_default_graph()

model_n1 = keras.models.load_model('my_GRU_model.h5', custom_objects={'AttentionWithContext':AttentionWithContext()})

model_n1.compile(Adam(lr=0.01), 'categorical_crossentropy', metrics=['accuracy'])
model_n1.load_weights("model_doc_tagging_attention_GRU_glove_vec_100d_new.h5")
#model_n1 = keras.models.load_model('my_GRU_model.h5', custom_objects={'AttentionWithContext':AttentionWithContext()})

#model_n1.compile(Adam(lr=0.01), 'categorical_crossentropy', metrics=['accuracy'])
#model_n1.load_weights("model_doc_tagging_attention_GRU_glove_vec_100d_new.h5")

file = open("tokenizer.pickle",'rb')
tokenizer = pickle.load(file)
print("len(tokenizer.word_index) + 1:",len(tokenizer.word_index) + 1)
file.close()

# API route
@app.route('/getfile', methods=['GET','POST'])
def getfile():
    if request.method == 'POST':
        labels = ['Bank Bailout', 'Budget', 'Credit Card', 'Family Finance', 
        'Job Benefits', 'Taxes'] 
        
        def cleanSentence(sentence):
             sentence_clean= re.sub("[^a-zA-Z0-9]"," ", str(sentence))
             sentence_clean = sentence_clean.lower()
             tokens = word_tokenize(sentence_clean)
             stop_words = set(stopwords.words("english"))
             sentence_clean_words = [w for w in tokens if not w in stop_words]
             return ' '.join(sentence_clean_words)

        def getTrainSequences(sentence, tokenizer,seq_maxlen):
            sent1 = tokenizer.texts_to_sequences(sentence)
            #sent_maxlen = max([len(s) for s in sent1])
            #seq_maxlen = max([sent_maxlen])
            return np.array(pad_sequences(sent1, maxlen=seq_maxlen))
        #Need pickel files of Tokenizer,Embedding weight and Glove matrix and freezed model

        # for secure filenames. Read the documentation.
        file = request.files['file']
        file.save(os.path.join('C:\\Users\\Atchyuta\\Downloads\\',file.filename))
        with open('C:\\Users\\Atchyuta\\Downloads\\'+ file.filename) as f:
            file_content = f.read().replace('\n', ' ')
        #Lines = file.readlines()
        #file_content= file.read().replace('\n', ' ')
        file_content=re.sub("\d+\.\d+ ", '',file_content)
        #print("in:",file_content)
        #data1=pd.DataFrame()
        #data1['file_content']=file_content
        dict1={"file_content":file_content}
        data1=pd.DataFrame(dict1,index=[0])
        #print(data1['file_content'])
        data1['clean_file_content'] = list(map(cleanSentence, data1['file_content']))
        print(data1['clean_file_content'])
        corpus_textn = '\n'.join((data1['clean_file_content']))
        sentences1 = corpus_textn.split('\n')
        train_data_need1 = getTrainSequences(sentences1, tokenizer,seq_maxlen=1680)
        print(train_data_need1.shape)
        #model_n1.load_weights("model_doc_tagging_attention_GRU_glove_vec_100d_new.h5")
        with graph.as_default():
            result=model_n1.predict(train_data_need1)
        print(result)
        result=result.argmax(axis=-1)
        label= labels[result[0]]
        print(label)
        #filename = secure_filename(file.filename) 
        test={"result":result}
        #keras.backend.clear_session()

        """# os.path.join is used so that paths work in every operating system
        file.save(os.path.join("wherever","you","want",filename))

        # You should use os.path.join here too.
        with open("wherever/you/want/filename") as f:
            file_content = f.read()"""

        return flask.jsonify({'predicted_tag': label })  


   
    return "txt file needed"

@app.route("/")
def index():
    return "Index API"


# HTTP Error handlers
@app.errorhandler(404)
def url_error(e):
    return f"""Wrong URL! <pre>{e}</pre>""", 404


@app.errorhandler(500)
def server_error(e):
    return (
        f"""An internal error occured: <pre>{e}</pre>. See logs for full stacktrace""",
        500,
    )


if __name__ == "__main__":
    # This is used when running locally
    app.run(host="0.0.0.0", debug=False)

len(tokenizer.word_index) + 1: 8216
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


0    agent silence customer right agent okay custom...
Name: clean_file_content, dtype: object
(1, 1680)


127.0.0.1 - - [04/Mar/2021 04:35:34] "POST /getfile HTTP/1.1" 200 -


[[4.5349399e-05 7.3971641e-08 9.9985337e-01 9.1207308e-05 6.5305089e-06
  3.4526777e-06]]
Credit Card


...

0    agent noise customer noise agent silence custo...
Name: clean_file_content, dtype: object
(1, 1680)
[[0.00886965 0.4247399  0.00158372 0.00095043 0.00784609 0.5560101 ]]
Taxes


127.0.0.1 - - [04/Mar/2021 04:38:27] "POST /getfile HTTP/1.1" 200 -


0    agent noise customer noise customer well uh kn...
Name: clean_file_content, dtype: object
(1, 1680)
[[3.3886121e-05 7.0710611e-01 1.8271609e-04 2.9222685e-01 1.8318297e-04
  2.6717989e-04]]
Budget


127.0.0.1 - - [04/Mar/2021 04:38:52] "POST /getfile HTTP/1.1" 200 -


0    agent noise topic today customer noise custome...
Name: clean_file_content, dtype: object
(1, 1680)
[[2.84697657e-04 9.66136754e-01 1.00715755e-04 3.23366523e-02
  1.07856991e-03 6.27010158e-05]]
Budget
